In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
#from time import time
#from tensorflow.python.keras.callbacks import TensorBoard 
#from keras.callbacks import TensorBoard

import tensorflow as tf

import cartpole_mod

Using TensorFlow backend.


In [2]:
# -*- coding: utf-8 -*-
EPISODES = 5

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        self.train_info = []
        #self.log_dir = './logs/log_{}'.format(time())
        #self.tensorboard = TensorBoard(log_dir='./logs/test', update_freq=128)

    """Huber loss for Q Learning
    References: https://en.wikipedia.org/wiki/Huber_loss
                https://www.tensorflow.org/api_docs/python/tf/losses/huber_loss
    """

    def _huber_loss(self, y_true, y_pred, clip_delta=1.0):
        error = y_true - y_pred
        cond  = K.abs(error) <= clip_delta

        squared_loss = 0.5 * K.square(error)
        quadratic_loss = 0.5 * K.square(clip_delta) + clip_delta * (K.abs(error) - clip_delta)

        return K.mean(tf.where(cond, squared_loss, quadratic_loss))

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate),
                      metrics=['accuracy'])
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        hist_batch = []
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            hist = self.model.fit(state, target, epochs=1, verbose=0)
            hist_batch.append([hist.history['acc'], hist.history['loss']])
        hist_batch = np.array(hist_batch)
        acc_loss = [np.mean(hist_batch[:,0]), np.mean(hist_batch[:,1])]
        self.train_info.append(acc_loss)
        #print(acc_loss)
        #print(self.train_info)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def get_history(self):       
        data = self.train_info
        self.train_info = []
        return data

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)



In [3]:

if __name__ == "__main__":
    
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.3
    set_session(tf.Session(config=config))

    
    
    env = gym.make('CartPole-v3')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-ddqn.h5")
    done = False
    batch_size = 32
    
    history = {}

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        history[e] = agent.get_history()
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-ddqn.h5")

W0724 19:43:32.825052 139813619951360 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 19:43:32.826357 139813619951360 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 19:43:32.829761 139813619951360 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0724 19:43:32.869781 139813619951360 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0724 19:43:32.882350 139813619951360 deprecation.py:323

episode: 0/5, score: 30, e: 1.0
episode: 1/5, score: 14, e: 0.88
episode: 2/5, score: 15, e: 0.75
episode: 3/5, score: 18, e: 0.63
episode: 4/5, score: 13, e: 0.55


history 변수에 episode마다 DNN 훈련 결과 저장  
훈련 결과는 Training Accuracy와 Training Loss  
각 episode에서 batch_size 만큼의 훈련 결과를 평균내어 한 번의 batch마다 한 번씩 저장  
Episode 1에서 10개의 training accuracy와 training loss가 저장됐으면 10번의 batch만큼 훈련된 것

In [11]:
history[4]

[[0.8125, 0.2901044345962873],
 [0.9375, 0.625334835441322],
 [1.0, 0.00396984377505305],
 [0.96875, 0.0013741646432094967],
 [1.0, 0.6321404570847236],
 [1.0, 0.821905322277928],
 [0.96875, 0.2304897342490797],
 [1.0, 0.4030779486689653],
 [1.0, 0.21764597333271496],
 [1.0, 0.19279532213933415],
 [1.0, 0.005478697363699325],
 [1.0, 0.618315787224688],
 [1.0, 0.0108691646859711]]